In [1]:
# add current dir to path
import os
import sys
nb_dir = os.path.abspath(os.path.split(os.getcwd())[0])
print(nb_dir)
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

c:\Users\harry\Dropbox (MIT)\code_ws_dropbox\itj_ws\integral_timber_joints\external\compas_fab_pychoreo\examples


In [2]:
from itj.parsing import parse_process, get_process_path, save_process_and_movements
from termcolor import cprint

In [3]:
process_file = 'twelve_pieces_process.json'
process = parse_process(process_file)
assembly = process.assembly # For convinence
toolchanger = process.robot_toolchanger

In [5]:
path = r'C:\Users\harry\Dropbox (MIT)\code_ws_dropbox\itj_ws\integral_timber_joints\external\compas_fab_pychoreo\examples\itj\data'
toolchanger.save_as_urdf(path, 1e-3)

file name:  L0_visual_0
file name:  L0_visual_1
file name:  L0_collision_0
file name:  L0_collision_1


In [4]:
from compas.robots import RobotModel, ToolModel
from integral_timber_joints.tools import Tool

isinstance(toolchanger, ToolModel)

True

In [6]:
for link in toolchanger.links:
    print(link.visual)
    for i, element in enumerate(link.collision):
        shape = element.geometry.shape
        print(i, element, shape.geometry.guid)
        print(element)
        # print(shape.geometry.guid)


[<compas.robots.model.link.Visual object at 0x000001F1A1823D48>, <compas.robots.model.link.Visual object at 0x000001F1A17FBC08>]
0 <compas.robots.model.link.Collision object at 0x000001F1A1808788> e23e6a6f-a779-4048-a26f-7acbf7e3fe82
1 <compas.robots.model.link.Collision object at 0x000001F1A1810548> 365a468d-f0f0-494a-b974-5ccba36fb949


In [24]:
from compas.files import URDF
from compas.robots import MeshDescriptor
# https://docs.python.org/3/library/xml.etree.elementtree.html
urdf = URDF.from_robot(toolchanger)
meshes = list(urdf.xml.root.iter('mesh'))
print(type(meshes[0]))
print(meshes[0].attrib)
# for m in meshes:
#     print(m.)

<class 'xml.etree.ElementTree.Element'>
{'filename': ''}


In [5]:
# all_movements = process.get_movements_by_beam_id('b0')
# save_process_and_movements(process_file, process, all_movements, overwrite=False,include_traj_in_process=False)

## Initial state

In [4]:
initial_state = process.initial_state

In [5]:
print(initial_state['robot'])

State: current frame: None | config: Configuration((21.400, -3.746, -3.620, -0.524, -0.524, 0.000, 0.000, 0.524, 0.000), (2, 2, 2, 0, 0, 0, 0, 0, 0), ('bridge1_joint_EA_X', 'robot11_joint_EA_Y', 'robot11_joint_EA_Z', 'robot11_joint_1', 'robot11_joint_2', 'robot11_joint_3', 'robot11_joint_4', 'robot11_joint_5', 'robot11_joint_6')) | attached to robot: False


In [6]:
env_models = process.environment_models

In [11]:
from compas_fab.robots import CollisionMesh

cm = CollisionMesh(env_models['e0'], 'e0')

In [13]:
cm.scale(1e-3)

## Get Gripper mesh / URDF

In [5]:
# gripper1 = process.gripper('g1')
# gripper1.to_urdf_file('g1.urdf', prettify=True)

In [6]:
from compas.geometry import Frame
tool = process.tool('g1')
tool.frame = Frame.worldXY()
tool_mesh = tool.draw_visual()

## Get States

In [23]:
process_file = 'twelve_pieces_process.json'
process = parse_process(process_file)

In [24]:
from integral_timber_joints.process import RoboticFreeMovement, RoboticLinearMovement, RoboticMovement

seq = assembly.sequence
actions = process.get_action_by_beam_id(seq[0])

for i, ac in enumerate(actions):
    print("======")
    print('{}): {}'.format(i,ac))
    for j, m in enumerate(ac.movements):
        print('---')
        print('#{} - priority {} | {}'.format(j, m.planning_priority, m.short_summary))
        print('Start conf: ', process.movement_has_start_robot_config(m))
        print('Start conf: ', process.movement_has_end_robot_config(m))
        try:
            print(m.allowed_collision_matrix)
        except:
            pass
        start_state = process.get_movement_start_state(m)
        end_state = process.get_movement_end_state(m)
        print(start_state['g1'].kinematic_config)
        print(start_state['g2'].kinematic_config)

0): Operator load Beam ('b0') for pickup
---
#0 - priority -1 | OperatorLoadBeamMovement(#A0_M0, Opeartor Load Beam to Pickup Location)
Start conf:  True
Start conf:  True
{'gripper_jaw_position': 0.8}
{'gripper_jaw_position': 0.8}
1): Pick PG1000 ('g2') from Storage
---
#0 - priority 0 | RoboticFreeMovement(#A1_M0, Free Move reach Storage Approach Frame of PG1000 ('g2'), to get tool., target conf 0, traj 0)
Start conf:  True
Start conf:  False
[]
{'gripper_jaw_position': 0.8}
{'gripper_jaw_position': 0.8}
---
#1 - priority 0 | RoboticLinearMovement(#A1_M1, Linear Advance to Storage Frame of PG1000 ('g2'), to get tool., target conf 1, traj 0)
Start conf:  False
Start conf:  True
[['robot', 'g2']]
{'gripper_jaw_position': 0.8}
{'gripper_jaw_position': 0.8}
---
#2 - priority -1 | RoboticDigitalOutput(#A1_M2, Toolchanger Lock PG1000 ('g2'))
Start conf:  True
Start conf:  True
{'gripper_jaw_position': 0.8}
{'gripper_jaw_position': 0.8}
---
#3 - priority -1 | RoboticDigitalOutput(#A1_M3, PG

In [6]:
process.get_movement_summary_by_beam_id(seq[0])

=====
Summary:
---
(0) OperatorLoadBeamMovement(#A0_M0, Opeartor Load Beam to Pickup Location) 
priority -1 | has start conf True | has end conf True
---
(1) RoboticFreeMovement(#A1_M0, Free Move reach Storage Approach Frame of PG1000 ('g2'), to get tool., target conf 0, traj 0) 
priority 0 | has start conf True | has end conf False | has traj None
---
(2) RoboticLinearMovement(#A1_M1, Linear Advance to Storage Frame of PG1000 ('g2'), to get tool., target conf 1, traj 0) 
priority 0 | has start conf False | has end conf True | has traj None
---
(3) RoboticDigitalOutput(#A1_M2, Toolchanger Lock PG1000 ('g2')) 
priority -1 | has start conf True | has end conf True
---
(4) RoboticDigitalOutput(#A1_M3, PG1000 ('g2') Open Gripper to release itself from storage pad.) 
priority -1 | has start conf True | has end conf True
---
(5) RoboticLinearMovement(#A1_M4, Linear Retract after getting PG1000 ('g2') from storage., target conf 0, traj 0) 
priority 0 | has start conf True | has end conf False

In [12]:
beam_id = seq[0]
all_movements = process.get_movements_by_beam_id(beam_id)
i=14
m = all_movements[i]
print(m.allowed_collision_matrix)

[]


In [16]:
movements.index(m)

1

In [9]:
print(process.get_movement_start_state(movements[2])['robot'])

State: current frame: Frame(Point(22744.870, 4645.293, 566.131), Vector(-1.000, -0.000, 0.000), Vector(-0.000, 1.000, -0.000)) | config: None | attached to robot: False | attached to joint: None


In [9]:
m_id = 11
m = movements[m_id]
print(type(m))
start_state = process.get_movement_start_state(m)
print(start_state.keys())

<class 'integral_timber_joints.process.movement.RoboticFreeMovement'>
dict_keys(['b0', 'b1', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19', 'b2', 'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29', 'b3', 'b30', 'b31', 'b32', 'b33', 'b34', 'b35', 'b36', 'b37', 'b38', 'b39', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'c1', 'c2', 'c3', 'c4', 'g1', 'g2', 'robot', 'tool_changer'])


In [10]:
m_id = 12
m = movements[m_id]
print(type(m))
start_state = process.get_movement_start_state(m)
print(start_state.keys())
print(start_state['robot'].to_data())
print(start_state['g1'].to_data())
print(start_state['tool_changer'].to_data())

<class 'integral_timber_joints.process.movement.RoboticLinearMovement'>
dict_keys(['b0', 'b1', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19', 'b2', 'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29', 'b3', 'b30', 'b31', 'b32', 'b33', 'b34', 'b35', 'b36', 'b37', 'b38', 'b39', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'c1', 'c2', 'c3', 'c4', 'g1', 'g2', 'robot', 'tool_changer'])
{'current_frame': Frame(Point(16038.051, 7049.996, 1844.357), Vector(0.000, 1.000, -0.000), Vector(-0.000, 0.000, 1.000)), 'kinematic_config': None, 'attached_to_robot': False, 'attached_to_joint': None}
{'current_frame': Frame(Point(23379.398, 4643.428, 712.425), Vector(0.000, -1.000, 0.000), Vector(-1.000, -0.000, -0.000)), 'kinematic_config': None, 'attached_to_robot': False, 'attached_to_joint': None}
{'current_frame': Frame(Point(16038.051, 7049.996, 1844.357), Vector(0.000, 1.000, -0.000), Vector(-0.000, 0.000, 1.000)), 'kinematic_config': None, 'attached_to_robot': True, 

In [60]:
# print(process.initial_state['b0'])
from compas.geometry import Frame

for object_id, object_state in process.initial_state.items():
    # Draw each object in the state dictionary
    # print (object_id, object_state)
    if object_id.startswith('b'):
        beam = process.assembly.beam(object_id)
        beam.current_frame = Frame.worldXY()
        meshes = beam.draw_state(object_state)

    # Tool objects
    # if object_id.startswith('c') or object_id.startswith('g'):
    #     tool = process.tool(object_id)
    #     meshes = tool.draw_state(object_state)

    # # Tool Changer
    # if object_id.startswith('t'):
    #     tool = process.robot_toolchanger
    #     meshes = tool.draw_state(object_state)

In [55]:
# TODO double check with Victor
print(process.environment_models)

{}


## Grasp

In [35]:
from compas.geometry import Transformation

beam_id = 'b1'
beam_ocf_from_gripper_tcp = Transformation.from_frame(process.assembly.get_beam_attribute(beam_id, "gripper_tcp_in_ocf"))
beam_ocf_from_gripper_tcp

Transformation([[1.0, -0.0, 0.0, 1541.2621765136737], [0.0, 0.0, -1.0, 100.0], [0.0, 1.0, 0.0, 50.0], [0.0, 0.0, 0.0, 1.0]])